<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/first_try/Notebooks/CompLingNoTrainerAllData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes

ConvAbuse is just the dictionary as a string lol

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

In [ ]:
filepaths = ["/content/ArMIS_train.json", "/content/ConvAbuse_train.json", "/content/HS-Brexit_train.json", "/content/MD-Agreement_train.json"]
dfs = [pd.read_json(filepath) for filepath in filepaths]
dfs = [df.transpose() for df in dfs]
dfs = [df.astype({"hard_label": int}, errors='raise') for df in dfs]

df_all = pd.concat(dfs)

# Dataset

In [84]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_all[["text", "hard_label", "soft_label"]])

In [85]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", do_lower_case=True)

In [86]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=240)#This is the longest that roberta tokenizer would go, dunno

tokenized_datasets = dataset.map(tokenize_function, batched=False)

  0%|          | 0/10431 [00:00<?, ?ex/s]

In [87]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)

In [88]:
tokenized_datasets.set_format("torch")#not sure if neccessary

In [89]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=4)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=4)

In [90]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", num_labels=2)

Some weights of the model checkpoint at lanwuwei/GigaBERT-v4-Arabic-and-English were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica

In [91]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters())

In [92]:
from transformers import get_scheduler
num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [93]:
import torch
from tqdm import tqdm_notebook
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
m = model.to(device)

In [94]:
import evaluate

#TODO do f1 with sklearn score


def evaluate_f1_score_huggingface(model, eval_dataloader):
  total = 0
  f1 = 0
  f1_metric = evaluate.load("f1")
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    N = predictions.shape[0]
    total+=1#the metric already gives the average, so no + batch_size
    f1 += f1_metric.compute(predictions=predictions, references=hard_labels, average = 'micro')["f1"]#micro must be used, otherwise smth wrong is returned, TODO look into that (00 00 is undefined)

  return f1/total


#slightly different to the pytorch implementations, not sure why TODO
def evaluate_ce_score(model, eval_dataloader, epsilon = 1e-12):
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    predictions = torch.clip(probabilities, epsilon, 1. - epsilon)                                      
    N = predictions.shape[0]
    total+=N
    ce += -torch.sum(soft_labels*torch.log(predictions+1e-9))
  return ce/total


def evaluate_bce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.BCELoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]["1"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    N = probabilities.shape[0]
    total +=N
    ce += criterion(probabilities[:,-1], soft_labels)
  return ce/total


def evaluate_ce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.CrossEntropyLoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)                          
    N = probabilities.shape[0]
    total+=N
    ce += criterion(probabilities, soft_labels)
  return ce/total



In [ ]:
criterion = torch.nn.CrossEntropyLoss()


In [95]:
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(tqdm_notebook(train_dataloader, 0)):
      #soft_label tracks both prob and the inverse prob lol
      #there is also index_level_0 in the dataset, not sure if needed
      inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]["1"].to(device)
      outputs = model(input_ids=inputs, attention_mask = attention_mask, labels=hard_labels)#NllLoss
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()

      running_loss += loss.item()
      if i % 10 == 9:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 10))
        running_loss = 0.0
      
    print(evaluate_bce_score_pytorch(model, eval_dataloader))

        
      


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/2347 [00:00<?, ?it/s]

[1,    10] loss: 1.150
[1,    20] loss: 0.738
[1,    30] loss: 0.799
[1,    40] loss: 0.753
[1,    50] loss: 0.606
[1,    60] loss: 0.556
[1,    70] loss: 0.633
[1,    80] loss: 0.479
[1,    90] loss: 0.506
[1,   100] loss: 0.820
[1,   110] loss: 0.704
[1,   120] loss: 0.514
[1,   130] loss: 0.567
[1,   140] loss: 0.549
[1,   150] loss: 0.636
[1,   160] loss: 0.653
[1,   170] loss: 0.733
[1,   180] loss: 0.591
[1,   190] loss: 0.691
[1,   200] loss: 0.569
[1,   210] loss: 0.640
[1,   220] loss: 0.580
[1,   230] loss: 0.393
[1,   240] loss: 0.587
[1,   250] loss: 0.689
[1,   260] loss: 0.652
[1,   270] loss: 0.541
[1,   280] loss: 0.506
[1,   290] loss: 0.553
[1,   300] loss: 0.577
[1,   310] loss: 0.586
[1,   320] loss: 0.641
[1,   330] loss: 0.603
[1,   340] loss: 0.751
[1,   350] loss: 0.500
[1,   360] loss: 0.650
[1,   370] loss: 0.590
[1,   380] loss: 0.563
[1,   390] loss: 0.580
[1,   400] loss: 0.785
[1,   410] loss: 0.466
[1,   420] loss: 0.609
[1,   430] loss: 0.713
[1,   440] 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/261 [00:00<?, ?it/s]

tensor(0.6069, device='cuda:0')


In [98]:
evaluate_ce_score(model, eval_dataloader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/261 [00:00<?, ?it/s]

tensor(0.6069, device='cuda:0')

In [99]:
evaluate_f1_score_huggingface(model, eval_dataloader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/261 [00:00<?, ?it/s]

0.7203065134099617